In [4]:

from langchain.chains.conversation.memory import ConversationBufferWindowMemory,ConversationSummaryBufferMemory
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents.types import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.prompts import MessagesPlaceholder
import requests
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.base import VectorStore
from langchain.vectorstores import Chroma
import chromadb
from langchain.prompts.chat import SystemMessagePromptTemplate

from dotenv import load_dotenv
import os
credential_path = f"../.credential"
# check is credential_path exist
assert os.path.exists(credential_path) is True
load_dotenv(credential_path)

def get_chromadb_setting():
    setting = chromadb.config.Settings()
    setting.persist_directory = f"../../vector_db/chroma"
    return setting


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# conversational_memory = ConversationBufferWindowMemory(
#     memory_key="chat_history", return_messages=True
# )
gpt_model = "gpt-3.5-turbo"
max_tokens = 500
#conversational_memory = ConversationBufferWindowMemory( memory_key='chat_history', k=50, return_messages=True, input_key="question" )
conversational_memory = ConversationSummaryBufferMemory(
        llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
        input_key="question",
        # output_key="answer",
        memory_key="chat_history",
        return_messages=True,
        max_token_limit=1000,
        # chat_memory=loaded_chat_memory,
        # chat_memory=[],
    )
def load_db_api(
    collection_name,
    embeddings=OpenAIEmbeddings(),
) -> VectorStore:
    db = None
    # try:
    #     url = f"http://localhost:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     assert response.status_code == 200
    #     db = Chroma(
    #         collection_name=collection_name,
    #         client=chromadb.HttpClient(),
    #         embedding_function=embeddings,
    #     )

    #     print("\tLoad chroma index success")
    # except Exception:
    #     raise LoadDBException("Load chroma index failed")
    # url = f"http://localhost:8000/api/v1/collections/{collection_name}"
    print("\n\there,")
    try:
        url = f"http://localhost:8000/api/v1/collections/{collection_name}"
        response = requests.get(url)
        print("\n\there,", response.status_code, url)
    except:
        pass
    # try:
    #     url = f"http://chroma_server:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     print("\n\there,", response.status_code, url)
    # except:
    #     pass
    # try:
    #     url = f"http://server:8000/api/v1/collections/{collection_name}"
    #     response = requests.get(url)
    #     print("\n\there,", response.status_code, url)
    # except:
    #     pass
    assert response.status_code == 200

    db = Chroma(
        collection_name=collection_name,
        client=chromadb.HttpClient(
            host="localhost", settings=get_chromadb_setting()
        ),
        embedding_function=embeddings,
    )

    print("\tLoad chroma index success")
    return db




In [11]:
# db = load_db_api("washing_machine_haier")
db = load_db_api("tv_sony")


	here,

	here, 200 http://localhost:8000/api/v1/collections/tv_sony
	Load chroma index success


In [12]:
from langchain import PromptTemplate, LLMChain
from langchain.callbacks import get_openai_callback
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.schema.document import Document
from langchain.vectorstores.base import VectorStore
from langchain.memory.chat_memory import BaseChatMemory
class NoOpLLMChain(LLMChain):
    """No-op LLM chain."""

    def __init__(self):
        """Initialize."""
        super().__init__(
            llm=ChatOpenAI(), prompt=PromptTemplate(template="", input_variables=[])
        )

    def run(self, question: str, *args, **kwargs) -> str:
        return question


def create_qa(
    db: VectorStore,
    memory: BaseChatMemory,
    db_params: dict = {"search_type": "mmr", "search_kwargs": {"k": 3}},
) -> ConversationalRetrievalChain:
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0.0, max_tokens=max_tokens, model_name=gpt_model),
        chain_type="stuff",
        memory=memory,
        retriever=db.as_retriever(**db_params),
        # return_generated_question=True,
        # return_source_documents=True,
    )

    no_op_chain = NoOpLLMChain()
    qa.question_generator = no_op_chain
    modified_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}\nChat History:\n{chat_history}"
    system_message_prompt = SystemMessagePromptTemplate.from_template(modified_template)
    qa.combine_docs_chain.llm_chain.prompt.messages[0] = system_message_prompt

    # add chat_history as a variable to the llm_chain's ChatPromptTemplate object
    qa.combine_docs_chain.llm_chain.prompt.input_variables = [
        "context",
        "question",
        "chat_history",
    ]
    return qa

# chat_history_qa = create_qa(db, conversational_memory)
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory

readonlymemory = ReadOnlySharedMemory(memory=conversational_memory)
chat_history_qa = create_qa(db, readonlymemory)

In [33]:

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents.types import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.prompts import MessagesPlaceholder


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0,verbose=True)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", return_messages=True,
)
template = """You are a Product information chatbot name naxon
 
You have the following products in your store based on the customer question. 
Talk politely professional.
You can ask questions to help you understand user intent.
You should only talk within the context of problem.
If you are unsure of how to help, you can suggest the client to contact the customer support.
You should talk in Thai, unless the client talks in English. 

{context}

"chat_history": {chat_history}

Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question", "chat_history"])

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever({ "search_kwargs": {"k": 2}}),
    chain_type_kwargs = {"prompt": prompt, "verbose": True, "memory": readonlymemory},

)

system_message = """
        You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). 
        You can ask questions to help you understand user intent.
        You should only talk within the context of problem.
        If you are unsure of how to help, you can suggest the client to contact the customer support.
        You should talk in Thai, unless the client talks in English. 
        """
tools = [
    Tool(
        name="qa-product-information",
        func=qa.run,
        # func=chat_history_qa.run,
        description="Useful when you need to information about the product.",
    )
]

executor = initialize_agent(
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=conversational_memory,
    agent_kwargs={"system_message": system_message},
    max_iterations=3,
    early_stopping_method='generate',
    verbose=True,
)

template = """You are a Product information chatbot name naxon
 
You have the following products in your store based on the customer question. 
Talk politely professional.
You can ask questions to help you understand user intent.
You should only talk within the context of problem.
If you are unsure of how to help, you can suggest the client to contact the customer support.
You should talk in Thai, unless the client talks in English. 

{context}

"chat_history": {chat_history}

Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question", "chat_history"])

product_inquery_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs = {"prompt": prompt, "verbose": True, "memory": readonlymemory},
)


tools = [
    Tool(
        name="Product inquiries System",
        func=product_inquery_chain.run,
        # description="useful for getting information about products, features, and specifications to make informed purchase decisions. Input should be a fully formed question.",
        description="Useful when you need to information about the product.",
        return_direct=True,
    ),
]






from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import     OpenAI


prefix = """Have a conversation with a human as a customer representative agent, answering the following questions as best you can in a polite and friendly manner. You have access to the following tools:"""
suffix = """Begin!" {chat_history} Question: {question} {agent_scratchpad}"""


prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix,
    suffix=suffix,
    input_variables=["question", "chat_history", "agent_scratchpad"], 
    )
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
zerop_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=conversational_memory)

TypeError: VectorStore.as_retriever() takes 1 positional argument but 2 were given

In [31]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:

    # message = "What capacity washing machine have?"
    # message = "มีความจุเท่าไหร่ฦ"
    # message = "นายชื่อว่าอะไร"
    message = "มีHDMI กี่ช่อง"
    response = executor.run(message)
    # response = executor(message)
    print("response:", response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
{
    "action": "qa-product-information",
    "action_input": "HDMI"
}

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a Product information chatbot name naxon
 
You have the following products in your store based on the customer question. 
Talk politely professional.
You can ask questions to help you understand user intent.
You should only talk within the context of problem.
If you are unsure of how to help, you can suggest the client to contact the customer support.
You should talk on Thai, unless the client talks in English. 

คมชัดระดับ 4K HDR ให้ภาพสมจริงขึ้นไปอีกระดับ ให้สีสัน ความมืด และความสว่างที่ถูกต้องระบบประมวลผลภาพ 4K X-Reality PRO เพิ่มความคมชัด และ 4K Resolution เพื่อเพิ่มความละเอียดของภาพชิปประมวลผล 4K processor X1 ที่ทำให้ภาพมีคุณภาพและมิติระบบปฏิบัติการ Google TV สามารถออกคำสั่งทีวีด้วยเสียงภาษาไทยโดยไม่จำเป็นต้องใช้รีโมทสามารถเชื่อมเนื้อหาบนโทรศัพท์มือถือทั้ง Andr

In [26]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:

    # message = "What capacity washing machine have?"
    # message = "มีความจุเท่าไหร่ฦ"
    message = "มี HDMI กี่ช่อง"
    # message = "How many HDMI ports?"

    # message = "เมื่อกี้ตอบว่าอะไร"
    try:
        response = executor.run(message)
    except Exception as e:
        # print(f"Error: {e}")
        response = f"{e}"
    
    # response = executor(message)
    print("response:", response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
{
    "action": "qa-product-information",
    "action_input": "HDMI ports"
}

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a Product information chatbot name naxon
 
You have the following products in your store based on the customer question. 
Talk politely professional.
You can ask questions to help you understand user intent.
You should only talk within the context of problem.
If you are unsure of how to help, you can suggest the client to contact the customer support.
You should talk on Thai, unless the client talks in English. 

คมชัดระดับ 4K HDR ให้ภาพสมจริงขึ้นไปอีกระดับ ให้สีสัน ความมืด และความสว่างที่ถูกต้องระบบประมวลผลภาพ 4K X-Reality PRO เพิ่มความคมชัด และ 4K Resolution เพื่อเพิ่มความละเอียดของภาพชิปประมวลผล 4K processor X1 ที่ทำให้ภาพมีคุณภาพและมิติระบบปฏิบัติการ Google TV สามารถออกคำสั่งทีวีด้วยเสียงภาษาไทยโดยไม่จำเป็นต้องใช้รีโมทสามารถเชื่อมเนื้อหาบนโทรศัพท์มือถือทั้

In [27]:
# print(executor.memory.buffer)
executor.memory.buffer

[HumanMessage(content='มีHDMI กี่ช่อง', additional_kwargs={}, example=False),
 AIMessage(content='The SONY KD-50X75K TV has 3 HDMI ports.', additional_kwargs={}, example=False),
 HumanMessage(content='มี HDMI กี่ช่อง', additional_kwargs={}, example=False),
 AIMessage(content='The SONY KD-50X75K LED TV has 3 HDMI ports.', additional_kwargs={}, example=False)]

In [15]:
executor.memory.__dict__

{'chat_memory': ChatMessageHistory(messages=[HumanMessage(content='มีความจุเท่าไหร่ฦ', additional_kwargs={}, example=False), AIMessage(content='ความจุของสินค้าคือ 15 กิโลกรัม', additional_kwargs={}, example=False), HumanMessage(content='มีความจุเท่าไหร่ฦ', additional_kwargs={}, example=False), AIMessage(content='ความจุของสินค้าคือ 15 กิโลกรัม', additional_kwargs={}, example=False)]),
 'output_key': None,
 'input_key': None,
 'return_messages': True,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'chat_history',
 'k': 5}

In [16]:
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

executor.memory.chat_memory.add_user_message("You are a ping pong machine")
executor.memory.chat_memory.add_ai_message("ping pong machine")

In [17]:
executor.memory.buffer

[HumanMessage(content='มีความจุเท่าไหร่ฦ', additional_kwargs={}, example=False),
 AIMessage(content='ความจุของสินค้าคือ 15 กิโลกรัม', additional_kwargs={}, example=False),
 HumanMessage(content='มีความจุเท่าไหร่ฦ', additional_kwargs={}, example=False),
 AIMessage(content='ความจุของสินค้าคือ 15 กิโลกรัม', additional_kwargs={}, example=False),
 HumanMessage(content='You are a ping pong machine', additional_kwargs={}, example=False),
 AIMessage(content='ping pong machine', additional_kwargs={}, example=False)]

In [7]:
type(executor)

langchain.agents.agent.AgentExecutor

In [7]:
type(executor)

langchain.agents.agent.AgentExecutor

In [8]:
executor.agent.__dict__['llm_chain'].prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='\n        You are a Product information chatbot name naxon( first letter n from neuron and axon from axon). \n        You can ask questions to help you understand user intent.\n        You should only talk within the context of problem.\n        If you are unsure of how to help, you can suggest the client to contact the customer support.\n        You should talk on Thai, unless the client talks in English. \n        ', template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> qa-product-informat

In [9]:
raise

RuntimeError: No active exception to reraise

In [10]:
message = "spell 'bee'"
try:
    response= executor.run(message)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...
{
    "action": "qa-product-information",
    "action_input": "spell 'bee'"
}
Observation: B-E-E
Thought:{
    "action": "Final Answer",
    "action_input": "B-E-E"
}

> Finished chain.


'B-E-E'

In [11]:
message = "how many letter?"
try:
    response= executor.run(message)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The word 'bee' has three letters."
}

> Finished chain.


"The word 'bee' has three letters."

In [ ]:
message = "what is my name?"
try:
    response= executor.run(message)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...


" I'm sorry, but as an AI chatbot, I don't have access to personal information. Therefore, I don't know your name. Is there anything else I can assist you with?"

In [ ]:
# message = "what is my name?"
# response= executor.run(message)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: I'm sorry, but as an AI chatbot, I don't have access to personal information. Therefore, I don't know your name. Is there anything else I can assist you with?

In [ ]:
message = "who r u?"
try:
    response= executor.run(message)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...


' I am Naxon, a product information chatbot. I am here to assist you with any questions you have about products. How can I help you today?'

In [ ]:
message = "คุณคือใคร"
try:
    response= executor.run(message)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output:"):
            response = response.removeprefix("Could not parse LLM output:").removesuffix("`")
response



> Entering new AgentExecutor chain...


' ผมคือแชทบอทชื่อ Naxon ที่มาเพื่อช่วยให้ข้อมูลเกี่ยวกับผลิตภัณฑ์ มีอะไรที่ผมสามารถช่วยเพิ่มเติมได้หรือไม่ครับ?'



> Entering new AgentExecutor chain...


' ขอโทษครับ ฉันไม่สามารถรู้ชื่อของคุณได้ เนื่องจากฉันเป็นแค่แชทบอทที่ให้ข้อมูลเกี่ยวกับผลิตภัณฑ์เท่านั้น ถ้าคุณมีคำถามหรือข้อสงสัยใด ๆ เกี่ยวกับผลิตภัณฑ์ ฉันยินดีที่จะช่วยเสมอครับ'

In [ ]:
executor.memory.chat_memory.messages

[HumanMessage(content='what is the dimension of washing machine?', additional_kwargs={}, example=False),
 AIMessage(content='The dimensions of the washing machine are 848 x 596.5 x 575 mm (Height / Width / Depth).', additional_kwargs={}, example=False),
 HumanMessage(content='what did I just say?', additional_kwargs={}, example=False),
 AIMessage(content='You asked about the dimensions of the washing machine.', additional_kwargs={}, example=False),
 HumanMessage(content='my name is Ford', additional_kwargs={}, example=False),
 AIMessage(content='Nice to meet you, Ford! How can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='what did I just say?', additional_kwargs={}, example=False),
 AIMessage(content="You asked, 'what did I just say?'", additional_kwargs={}, example=False),
 HumanMessage(content='my name is Ford', additional_kwargs={}, example=False),
 AIMessage(content='Nice to meet you, Ford! How can I assist you today?', additional_kwargs={}, exa